# Avaliação 01 - ENGG03

## Questão adaptada de [Galan et al. 2004](https://doi.org/10.1016/j.jprocont.2003.10.003)

Em um reator de neutralização, uma solução ácida (HCl) com vazão volumétrica $F_a = 1$ L/min e concentração inicial $C_{1,\text{in}} = 0.0012$ mol HCl/L é neutralizada por uma solução alcalina (NaOH + NaHCO3) com vazão $F_b = 0.82$ L/min e concentrações iniciais $C_{2,\text{in}} = 0.002$ mol NaOH/L e $C_{3,\text{in}} = 0.0025$ mol NaHCO3/L.
O comportamento dinâmico das concentrações dentro do reator, representado pelas variáveis $x_1$, $x_2$ e $x_3$, pode ser descrito pelo seguinte sistema de equações diferenciais:
\begin{align}
\frac{dx_1}{dt} &= \frac{Fa\cdot(C_{1,\text{in}}-x_1) - Fb\cdot x_1}{V}\\
\frac{dx_2}{dt} &= \frac{Fb\cdot(C_{2,\text{in}}-x_2) - Fa\cdot x_2}{V}\\
\frac{dx_3}{dt} &= \frac{Fb\cdot(C_{3,\text{in}}-x_3) - Fa\cdot x_3}{V}
\end{align}

Além disso, é possível determinar o pH da solução a partir do seguinte sistema de equações algébricas:

\begin{align*}
\text{pH} &= -\log_{10}(\xi) \\
\xi &= x_1 + \frac{K_e}{\xi} + \frac{K_e\cdot x_3}{K_e + K_d\cdot\xi} - x_2 - x_3
\end{align*}
na qual $K_e = 1\times 10^{-4}$ (mol/L)$^2$ e $K_d = 1\times 10^{-7}$ mol/L

A partir das informações dada, pede-se:

1.   No estado estacionário, as concentrações não variam com o tempo. Portanto, calcule os valores das concentrações $x_1$, $x_2$ e $x_3$ e o pH nessas condições.
2.   Assumindo a razão $u = \frac{F_b}{F_a}$, avalie o comportamento do pH para um intervalo de valores de $u$ no intervalo $[0,1]$.

  2.1. Dica: Utilize uma variação mínima de $\Delta u = 0.05$.

  2.2. Dica: Construa um gráfico de pH em função de $u$.

OBS: Comente seu código de maneira clara e objetiva deixando em evidência para que serve/objetivo de cada função, biblioteca e variáveis utilizadas.

## Solução dos problemas

### Concentrações no estado estacionário:

In [54]:
from scipy.optimize import fsolve
import numpy as np


# Definindo as Constantes
Fa = 1.0  # L/min (HCl)
Fb = 0.82  # L/min (NaOH + NaHCO3)

C1in = 0.0012  # mol HCl/L
C2in = 0.002  # mol NaOH/L
C3in = 0.0025  # mol NaHCO3/L

# Criando a matriz dos coeficientes
A = np.array(
    [
        [Fa + Fb, 0, 0],
        [0, Fb + Fa, 0],
        [0, 0, Fb + Fa],
    ]
)

# Criando o vetor das constantes
b = np.array([[Fa * C1in], [Fb * C2in], [Fb * C3in]])

# Resolvendo o sistema linear
sol = np.linalg.solve(A, b)

# Imprimindo os resultados
print("Concentrações no estado estacionário:")
X = [sol[0, 0], sol[1, 0], sol[2, 0]]
print("- Concentração de HCL:", X[0], "mol/L")
print("- Concentração de NaOH:", X[1], "mol/L")
print("- Concentração de NaHCO3:", X[2], "mol/L")
# Confere se ao aplicar ao aplicar os valore da solução o resultado é o vetor das constantes
print("Solução valida?", (A.dot(sol) == b).all())

Concentrações no estado estacionário:
- Concentração de HCL: 0.0006593406593406593 mol/L
- Concentração de NaOH: 0.0009010989010989011 mol/L
- Concentração de NaHCO3: 0.0011263736263736263 mol/L
Solução valida? True


### pH no estado estacionário:

In [55]:
def ph(V, X):
    # Variáveis
    pH = V[0]
    Csi = V[1]

    # Constantes
    x1 = X[0]
    x2 = X[1]
    x3 = X[2]
    Ke = 1e-4  # (mol/L)^2
    Kd = 1e-7  # mol/L

    # Equações
    e1 = pH + np.log10(Csi)
    e2 = x1 + (Ke / Csi) + ((Ke * x3) / (Ke + Kd * Csi)) - x2 - x3 - Csi

    return np.array([e1, e2])


# "Chute" inicial, talvez o pH final seja 7, afinal é um tanque neutralizador
Y0 = np.array([7, 1e-7])
Y = fsolve(ph, Y0, args=(X,))  # Resolve o sistema
print("pH no estado estacionário:", Y[0])
print("Teste da solução do sistema:", ph(Y, X))

pH no estado estacionário: 2.0052498273101382
Teste da solução do sistema: [0.00000000e+00 5.11569953e-15]
